# Language Modeling

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, sys, warnings, os, json
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

## Data Setup

In [2]:
data_dir = Path('../data')

In [3]:
onion_df = pd.read_csv(data_dir/'onion_headlines.csv')
onion_df.shape

(14363, 1)

In [5]:
from tokenizers.pre_tokenizers import *

In [6]:
tokenizers = BertPreTokenizer()